We try to copy "ClinicalTrials_Coding/COCTR_multidimensional_ranking-master/datasetPreparation/src/10_find_retrieved_nctIds_providing_single_query_and_datasetForEachClass.ipynb"

In [17]:
import pandas as pd
import os
import numpy as np
import networkx as nx

We first go through the trials retrieved per query by only the concept-based retrieval method. We will then form the extended trial set formed by adding the trials obtained through metapath-based similarity search method.

In [18]:
previous_datapath = 'C:/Users/roysoumya/Documents/ClinicalTrials_Coding/COCTR_multidimensional_ranking-master/rankedTrialsOnDifferentRelevancyBasedSchemes/1_pageRank'
output_datapath = "data/extended_retr_pagerank/"
extend_sim_data = 'data/extendedTrialsDetails_10thSept.csv'

In [19]:
extra_df = pd.read_csv(extend_sim_data)
extra_df['relevance'] = -1
extra_df.head()

,nct_id,query,brief_title,description,brief_title_conc_list,brief_summ_conc_list,total_subjects_affected,total_subjects_at_risk,relevance
0,NCT00000542,hypertension safe treatments_page_rank.csv,Antihypertensive and Lipid-Lowering Treatment ...,~ To determine if the combined incidence o...,C0018787;C0376495,C0027051;C0587184;C0012634;C0376495;C0597153;C...,0,0,-1
1,NCT00000542,treating people already having hypertension_pa...,Antihypertensive and Lipid-Lowering Treatment ...,~ To determine if the combined incidence o...,C0018787;C0376495,C0027051;C0587184;C0012634;C0376495;C0597153;C...,0,0,-1
2,NCT00000560,Nonvalvular atrial fibrillation_page_rank.csv,Beta-Blocker Evaluation in Survival Trial (BEST),~ To determine if addition of a beta-block...,C0220825,C0018801;C0456387;C0456387,0,0,-1
3,NCT00000620,treating people already having hypertension_pa...,Action to Control Cardiovascular Risk in Diabe...,~ The purpose of this study is to prevent ...,C3887460;C0011847,C0011849;C3887460;C3887460;C0005823;C0038454;C...,488,379287,-1
4,NCT00000877,antiretroviral therapy first time_pageRank.csv,Study of How Indinavir (an Anti-HIV Drug) and ...,~ The purpose of this study is to evaluate...,C0140575;C1446409;C0205160;C0012634,C4284232;C0140575;C0140575;C0140575;C0140575;C...,0,0,-1


In [20]:
extra_df.columns

Index(['nct_id', 'query', 'brief_title', 'description',
       'brief_title_conc_list', 'brief_summ_conc_list',
       'total_subjects_affected', 'total_subjects_at_risk', 'relevance'],
      dtype='object')

In [21]:
# We also append Popularity Values here
pop_data_readpath = "C:/Users/roysoumya/Documents/ClinicalTrials_Coding/TrialPopularityValues_10thSept.csv"
pop_data = pd.read_csv(pop_data_readpath)
extra_df = pd.merge(extra_df, pop_data, how='left', on= 'nct_id')
df['citationSum'] = df['citationSum'].fillna(0)
df['citationAvg'] = df['citationAvg'].fillna(0)
print(extra_df.columns)


Index(['nct_id', 'query', 'brief_title', 'description',
       'brief_title_conc_list', 'brief_summ_conc_list',
       'total_subjects_affected', 'total_subjects_at_risk', 'relevance',
       'citationSum', 'citationAvg'],
      dtype='object')


In [22]:
extra_df.rename(columns = {'total_subjects_affected':'tot_subj_affected', 'total_subjects_at_risk': 'tot_subj_risk' , 'brief_title_conc_list':'brief_title_concepts_list', 'brief_summ_conc_list': 'brief_summary_concepts_list'}, inplace = True) 

In [23]:
extra_df.columns

Index(['nct_id', 'query', 'brief_title', 'description',
       'brief_title_concepts_list', 'brief_summary_concepts_list',
       'tot_subj_affected', 'tot_subj_risk', 'relevance', 'citationSum',
       'citationAvg'],
      dtype='object')

using extra_df only for relevance computation.

In [24]:
#extra_df_rel = extra_df[["nct_id", "query", "relevance", "brief_title", "description", "brief_title_concepts_list", "brief_summary_concepts_list", "", ""]]
extra_df_rel = extra_df

In [25]:
csvFiles = sorted(os.listdir(previous_datapath))
csvFiles

['Dietary Therapy Epilepsies_page_rank.csv',
 'Early Parkinson disease treatment_page_rank.csv',
 'HIV infection Treatment naive_page_rank.csv',
 'HIV infection seronegativity_page_rank.csv',
 'Hypercholesterolemia safe treatments_page_rank.csv',
 'Nonvalvular atrial fibrillation_page_rank.csv',
 'Outcomes of cerebrovascular accident_page_rank.csv',
 'Treating Anemia, Iron-Deficiency in CKD patients_page_rank_15.csv',
 'already having Celiac Disease_page_rank.csv',
 'antiretroviral therapy first time_pageRank.csv',
 'constipation safe treatments_page_rank.csv',
 'dietary approaches for obesity treatment_page_rank.csv',
 'haemorrhage cure_page_rank.csv',
 'hypertension safe treatments_page_rank.csv',
 'low back pain therapy workout_pageRank.csv',
 'malnutrition in young children_page_rank.csv',
 'managing constipation in children_pageRank.csv',
 'out of hospital cardiac arrest_page_rank.csv',
 'postoperative delirium_pageRank.csv',
 'recommended anti-platelet doses for treating Coronary

In [26]:
df = pd.read_csv(os.path.join(previous_datapath, csvFiles[0]))
df.shape

(9, 26)

In [27]:
df.head()

,nct_id,relevance,brief_title,official_title,conditions_list,eligibility_criteria,description,keywords_list,Diseases,brief_title_concepts_list,...,tot_subj_affected,tot_subj_risk,Status,pubmed_ids,citationSum,citationAvg,briefTitlePageRankScore,briefTitleAndSummaryCombinedPageRankScore,inclusion,exclusion
0,NCT00504881,0,Brivaracetam as add-on Treatment in Adolescent...,"An International, Randomized, Double-blind, Pa...",Epilepsy;,Inclusion Criteria: - Subjects were ...,This study will compare the safety and efficac...,Epilepsy;Brivaracetam;Partial Onset Seizures;,Nervous System Diseases;,C1699861;C0087111;C0683278;C0014544,...,508.0,17760.0,Completed,24116853,18,18,0.122351,0.143706,"Subjects were aged from 16 to 70 years, inclus...",Subjects with a history or presence of Status ...
1,NCT00232596,0,Retigabine (Adjunctive Therapy) Efficacy and S...,"A Randomized, Double-Blind, Placebo-Controlled...",Epilepsy;Seizures;,Inclusion Criteria: - Diagnosis of r...,This Phase 3 study is being conducted to evalu...,Partial Seizures;Anticonvulsant;Complex Partia...,Nervous System Diseases;Pathological Condition...,C0530684;C2825951;C0036572;C0014544;C0087111;C...,...,1224.0,37960.0,Completed,22512894,11,11,0.105270,0.122911,Diagnosis of refractory epilepsy with simple o...,Clinically significant abnormalities on physic...
2,NCT00564915,1,RCT of the Efficacy of the Ketogenic Diet in t...,RCT of the Efficacy of the Ketogenic Diet in t...,Epilepsy;,Inclusion Criteria: - Diagnosis of e...,A study of the efficacy and tolerability of tw...,Epilepsy;,Nervous System Diseases;,C0259972;C0087111;C0014544,...,NaN,NaN,Completed,18456557,154,154,0.130384,0.111673,"Diagnosis of epilepsy for at least 2 years, re...",NaN
3,NCT01118949,0,Open-Label Study to Assess Lacosamide Safety a...,An Open-Label Pilot Study to Assess the Safety...,"Epilepsy;Seizures;Epilepsy, Generalized;",Inclusion Criteria: - Subject has a ...,The purpose is to assess the safety of Lacosam...,Primary Generalized Tonic-Clonic (PGTC) Seizur...,Nervous System Diseases;Pathological Condition...,C0494475;C0893761;C0014544;C0087111;C1522485;C...,...,145.0,931.0,Completed,28086164,2,2,0.101572,0.111632,Subject has a diagnosis of uncontrolled epilep...,Subject has a history of status epilepticus wi...
4,NCT01728077,0,"Evaluation of Long-term Safety, and Efficacy o...","An Open-label, Multicenter, Follow-up Study to...",Epilepsy;,Inclusion Criteria: - Subject is mal...,N01372 study is to evaluate the long-term safe...,Brivaracetam;Long-term Follow-up;Epilepsy;Part...,Nervous System Diseases;,C1699861;C0220825;C2825951;C0087111;C0014544,...,108.0,1092.0,Completed,26899665,0,0,0.114814,0.108901,Subject is male or female and 16 years or olde...,"Severe medical, neurological, or psychiatric d..."


In [28]:
df.columns

Index(['nct_id', 'relevance', 'brief_title', 'official_title',
       'conditions_list', 'eligibility_criteria', 'description',
       'keywords_list', 'Diseases', 'brief_title_concepts_list',
       'official_title_concepts_list', 'brief_summary_concepts_list',
       'conditions_list_concepts_list', 'eligibilities_concepts_list',
       'keywords_list_concepts_list', 'CompletionDate', 'tot_subj_affected',
       'tot_subj_risk', 'Status', 'pubmed_ids', 'citationSum', 'citationAvg',
       'briefTitlePageRankScore', 'briefTitleAndSummaryCombinedPageRankScore',
       'inclusion', 'exclusion'],
      dtype='object')

In [29]:
def applyPageRank(df1,query):
    G1 = nx.Graph()
    G2 = nx.Graph()

    i = 0
    for nct1 in df1.nct_id:
        j = 0
    #         print(nct1)
        try:
            iConceptsBriefTitle = set(df1.iloc[i]['brief_title_concepts_list'].split(";"))
    #             print(iConceptsBriefTitle)
        except:
            iConceptsBriefTitle = set()
        try:
            iConceptsBriefSummary = set(df1.iloc[i]['brief_summary_concepts_list'].split(";"))
        except:
            iConceptsBriefSummary = set()
        try:
            iUnionTitleSummary = iConceptsBriefTitle.union(iConceptsBriefSummary)
        except:
            iUnionTitleSummary = set()

        for nct2 in df1.nct_id:
            try:
                jConceptsBriefTitle = set(df1.iloc[j]['brief_title_concepts_list'].split(";"))
            except:
                jConceptsBriefTitle = set()

            try:
                jConceptsBriefSummary = set(df1.iloc[j]['brief_summary_concepts_list'].split(";"))
            except:
                jConceptsBriefSummary = set()

            try:
                jUnionTitleSummary = jConceptsBriefTitle.union(jConceptsBriefSummary)
            except:
                jUnionTitleSummary = ()

            try:
                score1 = len(iConceptsBriefTitle.intersection(jConceptsBriefTitle))/min(len(iConceptsBriefTitle), len(jConceptsBriefTitle))
            except:
                score1 = 0
            try:
                score2 = len(iUnionTitleSummary.intersection(jUnionTitleSummary))/min(len(iUnionTitleSummary), len(jUnionTitleSummary))
            except:
                score2 = 0
            #             print(score)
    #             print(score1, score2)
            G1.add_edge(i, j, weight=score1)
            G2.add_edge(i, j, weight=score2)
            j += 1
        i += 1

    print(G1, G2)
    pr1 = nx.pagerank_numpy(G1, alpha=0.9)
    pr2 = nx.pagerank_numpy(G2, alpha=0.9)

    pageRankList1 = []
    pageRankList2 = []

    for key in pr1.keys():
        pageRankScore1 = pr1[key]
        pageRankScore2 = pr2[key]

        pageRankList1.append(pageRankScore1)
        pageRankList2.append(pageRankScore2)

    df1['briefTitlePageRankScore'] = pageRankList1
    df1['briefTitleAndSummaryCombinedPageRankScore'] = pageRankList2
    df1 = df1.sort_values(by=['briefTitleAndSummaryCombinedPageRankScore'], ascending=False)
    df1.to_csv(os.path.join(output_datapath, query + "_ext_page_rank_" + str(df1.shape[0]) + ".csv"), index=False)

In [30]:
for file in csvFiles:
    df1 = pd.read_csv(os.path.join(previous_datapath, file))
    df2 = df1[['nct_id', 'brief_title', 'description',
       'brief_title_concepts_list', 'brief_summary_concepts_list',
       'tot_subj_affected', 'tot_subj_risk', 'relevance', 'citationSum',
       'citationAvg']]
    
    extra_trials = extra_df_rel[extra_df_rel["query"] == file]
    print(file, extra_trials.shape)
    extra_trials = extra_trials[['nct_id', 'brief_title', 'description',
       'brief_title_concepts_list', 'brief_summary_concepts_list',
       'tot_subj_affected', 'tot_subj_risk', 'relevance', 'citationSum',
       'citationAvg']]
    
    tot_trials = pd.concat([df2, extra_trials])
    query = file.split("_")[0]
    applyPageRank(tot_trials,query=query)
  

Dietary Therapy Epilepsies_page_rank.csv (11, 11)
 
Early Parkinson disease treatment_page_rank.csv (22, 11)
 
HIV infection Treatment naive_page_rank.csv (13, 11)
 
HIV infection seronegativity_page_rank.csv (33, 11)
 
Hypercholesterolemia safe treatments_page_rank.csv (7, 11)
 
Nonvalvular atrial fibrillation_page_rank.csv (91, 11)
 
Outcomes of cerebrovascular accident_page_rank.csv (145, 11)
 
Treating Anemia, Iron-Deficiency in CKD patients_page_rank_15.csv (1, 11)
 
already having Celiac Disease_page_rank.csv (5, 11)
 
antiretroviral therapy first time_pageRank.csv (29, 11)
 
constipation safe treatments_page_rank.csv (12, 11)
 
dietary approaches for obesity treatment_page_rank.csv (30, 11)
 
haemorrhage cure_page_rank.csv (31, 11)
 
hypertension safe treatments_page_rank.csv (28, 11)
 
low back pain therapy workout_pageRank.csv (18, 11)
 
malnutrition in young children_page_rank.csv (26, 11)
 
managing constipation in children_pageRank.csv (23, 11)
 
out of hospital cardiac arr